In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator( rescale = 1./255,
                                    validation_split=0.2,
                                    rotation_range = 20,
                                    horizontal_flip = True,
                                    vertical_flip=True,
                                    fill_mode = 'nearest'
                                  )

val_datagen = ImageDataGenerator(rescale = 1./255, 
                                 validation_split=0.2,
                                )
test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:
train_data = train_datagen.flow_from_directory('ckplus/train',
                                                target_size = (100,100),
                                                class_mode = 'categorical',
                                                batch_size = 64,
                                                subset='training'
                                              )

val_data = val_datagen.flow_from_directory('ckplus/train',
                                            target_size = (100,100),
                                            class_mode = 'categorical',
                                            batch_size = 64,
                                            subset='validation'
                                          )

test_data = test_datagen.flow_from_directory('ckplus/test',
                                              target_size = (100,100),
                                              class_mode = 'categorical',
                                              batch_size = 64
                                            )

Found 1372 images belonging to 7 classes.
Found 340 images belonging to 7 classes.
Found 430 images belonging to 7 classes.


In [4]:
train_steps = train_data.n//train_data.batch_size
val_steps = val_data.n//val_data.batch_size
test_steps = test_data.n//test_data.batch_size

In [5]:
# Resnet50 model
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(100,100,3))
# base_model = VGG16(include_top=False, weights='imagenet', input_shape=(100,100,3))

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False
    
# unfreeze the last 3 layers
for layer in base_model.layers[-3:]:
    layer.trainable = True
    
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # Assuming 7 classes for emotions

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 512)               16777728  
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                      

In [6]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
checkpoint = ModelCheckpoint('emotion_model.h5', save_best_only=True)

# Training the model
history = model.fit(train_data,
                    steps_per_epoch=train_steps,
                    epochs=200,
                    validation_data=val_data,
                    validation_steps=val_steps,
                    callbacks=[reduce_lr, checkpoint]
                   )

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_data, steps=test_steps)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')



Epoch 1/200


21/21 [==============================] - ETA: 0s - loss: 2.1725 - accuracy: 0.2691

c:\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


21/21 [==============================] - 83s 3s/step - loss: 2.1725 - accuracy: 0.2691 - val_loss: 5.3889 - val_accuracy: 0.2188 - lr: 0.0010
Epoch 2/200
21/21 [==============================] - 58s 3s/step - loss: 1.8638 - accuracy: 0.3601 - val_loss: 4.4237 - val_accuracy: 0.2313 - lr: 0.0010
Epoch 3/200
21/21 [==============================] - 55s 3s/step - loss: 1.7656 - accuracy: 0.3670 - val_loss: 4.8789 - val_accuracy: 0.2281 - lr: 0.0010
Epoch 4/200
21/21 [==============================] - 55s 3s/step - loss: 1.6426 - accuracy: 0.4358 - val_loss: 4.9206 - val_accuracy: 0.2156 - lr: 0.0010
Epoch 5/200
21/21 [==============================] - 55s 3s/step - loss: 1.5191 - accuracy: 0.4633 - val_loss: 4.5585 - val_accuracy: 0.1937 - lr: 0.0010
Epoch 6/200
21/21 [==============================] - 59s 3s/step - loss: 1.4393 - accuracy: 0.4870 - val_loss: 3.7756 - val_accuracy: 0.1906 - lr: 0.0010
Epoch 7/200
21/21 [==============================] - 69s 3s/step - loss: 1.3769 - accura